# 数据处理

In [ ]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"

! pip install langchain_community
! pip install langchain
! pip install transformers
! pip install unstructured
! pip install markdown
! pip install markdownify
! pip install sentence-transformers
! pip install clickhouse-connect

## 查看目录中文档的最大长度

In [ ]:
import os

root_dir = "/root/jd_docs"
size = 0
for root, dirs, files in os.walk(root_dir, followlinks=False):
    for file in files:
        path = os.path.join(root, file)
        if os.path.getsize(path) > size:
            size = os.path.getsize(path)
size

## 去标签实验

In [ ]:
from markdownify import markdownify as md
import re


def is_html(content):
    # content = content.read(15)  # 读取文件开头的15个字节
    if content.startswith('<'):
        return True
    else:
        return False


def replace_unicode(match):
    code_point = int(match.group(1), 16)
    return chr(code_point)


# f = open("/root/jd_docs/account-assets/withdrawal.md")
f = open("/root/jd_docs/application-load-balancer/TLSSecurityPolicy-management.md")
lines = f.read()


# 替换unicode字符为标签
result = re.sub(r"\\[uU]([0-9a-fA-F]{4})", replace_unicode, lines)


# 如果为html 格式，转换为markdown
if is_html(result):
    result = md(result)
    print(result)

# 去html标签
pattern = re.compile(r'<[^>]+>', re.S)
result = pattern.sub(' ', result)

print(result.replace('\\t', '').replace('\\n', ''))
f.close()

## 文本分割，拆分为定长文本


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"

root_dir = "/root/jd_docs"
# for root, dirs, files in os.walk(root_dir, followlinks=False):
#     for file in files:
#         path = os.path.join(root, file)
#         loader = UnstructuredMarkdownLoader(path)
#         data = loader.load()
#         print(data)


loader = DirectoryLoader(
    '/root/jd_docs', glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()

# splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
# chunked_docs = splitter.split_documents(docs)

## 存入向量数据库

In [ ]:
import langchain_community.vectorstores.clickhouse as clickhouse
from langchain.embeddings import HuggingFaceEmbeddings
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="moka-ai/m3e-large", model_kwargs=model_kwargs)

settings = clickhouse.ClickhouseSettings(
    table="jd_docs_m3e", username="default", password="Git785230", host="10.0.1.94")
docsearch = clickhouse.Clickhouse.from_documents(
    docs, embeddings, config=settings)